In [1]:
import boto3

s3 = boto3.resource('s3')

## To verify access to AWS

In [ ]:
for bucket in s3.buckets.all():
    print(bucket.name)

## Text generation using Amazon Titan 

In [2]:
bedrock = boto3.client(service_name='bedrock',
                       region_name='us-west-2')

In [3]:
bedrock.list_foundation_models()

{'ResponseMetadata': {'RequestId': '12533809-0988-4d84-a9cb-8c57a4cbecdc',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Mon, 05 May 2025 16:10:13 GMT',
   'content-type': 'application/json',
   'content-length': '57809',
   'connection': 'keep-alive',
   'x-amzn-requestid': '12533809-0988-4d84-a9cb-8c57a4cbecdc'},
  'RetryAttempts': 0},
 'modelSummaries': [{'modelArn': 'arn:aws:bedrock:us-west-2::foundation-model/amazon.titan-tg1-large',
   'modelId': 'amazon.titan-tg1-large',
   'modelName': 'Titan Text Large',
   'providerName': 'Amazon',
   'inputModalities': ['TEXT'],
   'outputModalities': ['TEXT'],
   'responseStreamingSupported': True,
   'customizationsSupported': [],
   'inferenceTypesSupported': ['ON_DEMAND'],
   'modelLifecycle': {'status': 'ACTIVE'}},
  {'modelArn': 'arn:aws:bedrock:us-west-2::foundation-model/amazon.nova-premier-v1:0:8k',
   'modelId': 'amazon.nova-premier-v1:0:8k',
   'modelName': 'Nova Premier',
   'providerName': 'Amazon',
   'inputModalities': [

In [ ]:
!aws sts get-caller-identity

In [4]:
# to get info about a particular model rather than for all which was given in above cell 
bedrock.get_foundation_model(modelIdentifier='amazon.titan-text-express-v1')

{'ResponseMetadata': {'RequestId': '65f382fd-2668-4ffc-b01c-d6b407f8b99e',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Mon, 05 May 2025 16:10:20 GMT',
   'content-type': 'application/json',
   'content-length': '524',
   'connection': 'keep-alive',
   'x-amzn-requestid': '65f382fd-2668-4ffc-b01c-d6b407f8b99e'},
  'RetryAttempts': 0},
 'modelDetails': {'modelArn': 'arn:aws:bedrock:us-west-2::foundation-model/amazon.titan-text-express-v1',
  'modelId': 'amazon.titan-text-express-v1',
  'modelName': 'Titan Text G1 - Express',
  'providerName': 'Amazon',
  'inputModalities': ['TEXT'],
  'outputModalities': ['TEXT'],
  'responseStreamingSupported': True,
  'customizationsSupported': [],
  'inferenceTypesSupported': ['ON_DEMAND'],
  'modelLifecycle': {'status': 'ACTIVE'}}}

Great, now we want to see how to make an API call 

In [5]:
question = 'What is the capital of France'

In order to **invoke the model** I need to access the **bedrock runtime service.** So something to keep in mind is slightly different than other boto services is the service name. **Bedrock** is basically just information about Amazon bedrock. Like what models are available, what models do I have access to, what services are available, etc. when I actually want to run one of these models, the service name is **bedrock-runtime.**  

In [6]:
bedrock_runtime = boto3.client(region_name='us-west-2', 
                               service_name='bedrock-runtime')

Now, something to note is when you actually invoke these models, it expects a JSON request body. And depending on the model the actual configuration of that JSON may be slightly different, which looks like the following for the Amazon Titan model:

        {
            "inputText": string,
            "testGenerationConfig":{
                "temperature": float,
                "topP": float,
                "maxTokenCount": int,
                "stopSequences": [string]
            }
        }

In [7]:
import json
body = json.dumps({'inputText': question})

In [8]:
body

'{"inputText": "What is the capital of France"}'

In [9]:
type(body)

str

In [10]:
response = bedrock_runtime.invoke_model(body=body,
                                        modelId='amazon.titan-text-express-v1')
# this is the actual API request to that particular model 

In [11]:
response

{'ResponseMetadata': {'RequestId': '2a99af4d-6c5a-4ebc-902e-8dfdfb053b9b',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Mon, 05 May 2025 16:10:37 GMT',
   'content-type': 'application/json',
   'content-length': '137',
   'connection': 'keep-alive',
   'x-amzn-requestid': '2a99af4d-6c5a-4ebc-902e-8dfdfb053b9b',
   'x-amzn-bedrock-invocation-latency': '742',
   'x-amzn-bedrock-output-token-count': '10',
   'x-amzn-bedrock-input-token-count': '6'},
  'RetryAttempts': 0},
 'contentType': 'application/json',
 'body': <botocore.response.StreamingBody at 0x1ba23172350>}

And then if we check out what the response is it's this large response object. And you'll notice I need to actually grab and read from the body because the body itself has this response dot streaming body object. So I can't directly grab it just off the response. It has a lot of information on things like how many tokens did you pass in, etc..

In [12]:
response_body = json.loads(response.get('body').read())

In [13]:
response_body

{'inputTextTokenCount': 6,
 'results': [{'tokenCount': 10,
   'outputText': '\nParis is the capital city of France.',
   'completionReason': 'FINISH'}]}

In [14]:
response_body['results']

[{'tokenCount': 10,
  'outputText': '\nParis is the capital city of France.',
  'completionReason': 'FINISH'}]

In [15]:
print(response_body['results'][0]['outputText'])


Paris is the capital city of France.


In [16]:
question_2 = "Tell me a story about a magic forest."

In [17]:
body_2 = json.dumps({'inputText': question_2,
                     'textGenerationConfig': {'temperature': 0,
                                              'topP': 0.01,
                                              'maxTokenCount': 512
                                              }
                    })

In [18]:
body_2

'{"inputText": "Tell me a story about a magic forest.", "textGenerationConfig": {"temperature": 0, "topP": 0.01, "maxTokenCount": 512}}'

In [19]:
response_2 = bedrock_runtime.invoke_model(body=body_2,
                                          modelId = 'amazon.titan-text-express-v1')

In [20]:
response_2

{'ResponseMetadata': {'RequestId': '3878d282-9b79-4e1a-88f9-ab1d49ff0354',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Mon, 05 May 2025 16:11:16 GMT',
   'content-type': 'application/json',
   'content-length': '2568',
   'connection': 'keep-alive',
   'x-amzn-requestid': '3878d282-9b79-4e1a-88f9-ab1d49ff0354',
   'x-amzn-bedrock-invocation-latency': '24887',
   'x-amzn-bedrock-output-token-count': '512',
   'x-amzn-bedrock-input-token-count': '9'},
  'RetryAttempts': 0},
 'contentType': 'application/json',
 'body': <botocore.response.StreamingBody at 0x1ba231b4370>}

In [21]:
response_body_2 = json.loads(response_2.get('body').read())

In [22]:
print(response_body_2['results'][0]['outputText'])


Once upon a time, there was a magical forest filled with talking animals and enchanted creatures. A young girl named Lily discovered the forest while exploring the woods near her home. She met a wise old owl who told her about the forest's secrets and the dangers that lurked within.

Lily was amazed by the beauty of the forest and the creatures that lived there. She met a talking rabbit named Hoppy, a playful squirrel named Nutty, and a beautiful butterfly named Flutterby. They showed her around the forest and taught her about the different plants and animals that lived there.

One day, Lily stumbled upon a group of fairies who were in trouble. They had lost their magic wand and were unable to perform their magic. Lily offered to help them find it and they agreed.

Together, they searched the forest high and low until they found the wand in a hidden cave. The fairies were overjoyed and thanked Lily for her help. They invited her to stay with them in their magical kingdom and Lily acce

In [23]:
saved_story = response_body_2['results'][0]['outputText']

In [24]:
print(saved_story)


Once upon a time, there was a magical forest filled with talking animals and enchanted creatures. A young girl named Lily discovered the forest while exploring the woods near her home. She met a wise old owl who told her about the forest's secrets and the dangers that lurked within.

Lily was amazed by the beauty of the forest and the creatures that lived there. She met a talking rabbit named Hoppy, a playful squirrel named Nutty, and a beautiful butterfly named Flutterby. They showed her around the forest and taught her about the different plants and animals that lived there.

One day, Lily stumbled upon a group of fairies who were in trouble. They had lost their magic wand and were unable to perform their magic. Lily offered to help them find it and they agreed.

Together, they searched the forest high and low until they found the wand in a hidden cave. The fairies were overjoyed and thanked Lily for her help. They invited her to stay with them in their magical kingdom and Lily acce

In [25]:
question_3 = "Tell me a very creative story about a magic forest."

In [32]:
# let's increase topP and temperature
body_3 = json.dumps({'inputText': question_3,
                     'textGenerationConfig': {'temperature': 1,
                                              'topP': 1,
                                              'maxTokenCount': 1024
                                              }
                    })


In [33]:
response_3 = bedrock_runtime.invoke_model(body=body_3,
                                          modelId = 'amazon.titan-text-express-v1')

In [34]:
response_body_3 = json.loads(response_3.get('body').read())

In [35]:
print(response_body_3['results'][0]['outputText'])


In the heart of a grand rainforest full of sparkling fauna, trees shimmer in the light. Welcome to Magic Tree World, where nature thrives with mysterious powers. You arrive at nightfall, a shining golden gate welcoming your sight.

You soon meet Finn, your companion. You spend your days with him, exploring the magical forest you enter very quickly. Over time, both children come to love this magical world.

One afternoon, Finn and you rush to explore the beautiful forest corner. They dance happily amid the gorgeous red, orange, and blue leaves. You suddenly heard something strange. It's Chacha, who invites you to join her in his home. He's your circus friend and now heads away on vacation.

Chacha's home is a large yurt decorated with beads and feathers. When you enter, you find Chacha's mother busy cooking apples. He kindly invites you to eat the apples with cinnamon. But suddenly everything begins to dance. Singing faeries invite the two magically into their wonderland. Maria is stun

In [36]:
len(response_body_3['results'][0]['outputText'])

1350

In [37]:
len(saved_story)

2443